## Assessing the S&P 500

### Introduction

This Jupyter Notebook aims to provide a brief and simple view of assessing the **S&P 500 Index**. The S&P 500 is a market-cap weighted index consisting of 500 of the largest publicly traded companies in the United States. The companies included in this index are selected based on their market capitalization, liquidity, and industry representation, which gives a comprehensive snapshot of the U.S. stock market's performance.

The S&P 500 is widely regarded as one of the best representations of the U.S. equity market and is often used as a benchmark by investors to compare their performance against. If an investor’s portfolio is performing similarly to or better than the S&P 500, it suggests that their investments are on par with the broader market. Conversely, underperforming the index may signal a need for strategy reassessment.

### Part 1: Retrieving the S&P 500 Constituents

The S&P 500 index is updated periodically, so the exact list of companies in the index may change over time. However, we can obtain the list of current S&P 500 companies from publicly available sources such as **Wikipedia**, which is regularly updated with the latest constituents.

Each company in the S&P 500 is assigned a ticker symbol, which is used to uniquely identify it on stock exchanges. For example, the ticker for Apple Inc. is **AAPL**, while the ticker for Microsoft is **MSFT**.

Once we retrieve the list of tickers, we can use these symbols to fetch market data and financial information for the companies included in the S&P 500.
In addition, we will examine the sector segments of the S&P 500 constituents in terms of their proportions, as well as analyze their subindustry breakdowns. This will help us visualize the distribution of companies across sectors and subindustries, offering a clearer picture of the market composition. 

This Notebook was inspired primarily by the tutorial 'Stock Analysis with Yahoo Finance, Plotly, Pandas, and Pandas_Ta' by Adi Dror, which can be found on https://www.linkedin.com/pulse/stock-analysis-yahoo-finance-plotly-pandas-pandasta-adi-dror-udg3f/

In [70]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Fetch the page content
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

# Parse the HTML
soup = BeautifulSoup(response.text, "html.parser")

# Find the first table with the S&P 500 list
table = soup.find("table", {"class": "wikitable"})

# Convert to DataFrame
SP_500_DF = pd.read_html(str(table))[0]

print(SP_500_DF.head())


  Symbol             Security             GICS Sector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories             Health Care   
3   ABBV               AbbVie             Health Care   
4    ACN            Accenture  Information Technology   

                GICS Sub-Industry    Headquarters Location  Date added  \
0        Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1               Building Products     Milwaukee, Wisconsin  2017-07-26   
2           Health Care Equipment  North Chicago, Illinois  1957-03-04   
3                   Biotechnology  North Chicago, Illinois  2012-12-31   
4  IT Consulting & Other Services          Dublin, Ireland  2011-07-06   

       CIK      Founded  
0    66740         1902  
1    91142         1916  
2     1800         1888  
3  1551152  2013 (1888)  
4  1467373         1989  


/var/folders/gm/fn54mj5d4b51_7l5j667v4n00000gn/T/ipykernel_1423/475999257.py:17: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



In [71]:
SECTOR_DISTRIBUTION=SP_500_DF['GICS Sector'].value_counts(normalize=True)
SECTOR_DISTRIBUTION

GICS Sector
Industrials               0.155070
Financials                0.145129
Information Technology    0.137177
Health Care               0.121272
Consumer Discretionary    0.099404
Consumer Staples          0.075547
Utilities                 0.061630
Real Estate               0.061630
Materials                 0.055666
Communication Services    0.043738
Energy                    0.043738
Name: proportion, dtype: float64

In [72]:
import plotly.express as px 

px.pie(SECTOR_DISTRIBUTION.reset_index(), values='proportion',names='GICS Sector',title='GICS Sector Breakdown')

In [73]:
SECTOR_INDUSTRY_DISTRIBUTION=SP_500_DF[['GICS Sector','GICS Sub-Industry']].value_counts(normalize=True)
SECTOR_INDUSTRY_DISTRIBUTION

GICS Sector             GICS Sub-Industry                           
Health Care             Health Care Equipment                           0.035785
Utilities               Electric Utilities                              0.029821
Information Technology  Semiconductors                                  0.027833
Industrials             Industrial Machinery & Supplies & Components    0.027833
                        Aerospace & Defense                             0.023857
                                                                          ...   
                        Heavy Electrical Equipment                      0.001988
                        Passenger Ground Transportation                 0.001988
Information Technology  Technology Distributors                         0.001988
Materials               Commodity Chemicals                             0.001988
Utilities               Water Utilities                                 0.001988
Name: proportion, Length: 127, dtype: fl

In [97]:
px.pie(SECTOR_INDUSTRY_DISTRIBUTION.reset_index(), values='proportion',names='GICS Sub-Industry',title='GICS Sub-Industry',facet_col='GICS Sector', facet_col_wrap=3,height=800, width=1300)

In [75]:
 #Create treemap
fig = px.treemap(
    SECTOR_INDUSTRY_DISTRIBUTION.reset_index(),
    path=[px.Constant("SP-500"), 'GICS Sector', 'GICS Sub-Industry'],
    values='proportion',
    height=1000,
    width=1000
)

# Adjust layout for better text visibility
fig.update_layout(
    font=dict(size=9),  # Increase font size
    uniformtext=dict(minsize=9, mode='show'),
    yaxis=dict(automargin=True)
)

# Display the chart
fig.show()

# 📊 **Downloading S&P 500 Data from Yahoo Finance**

In this guide, we'll walk through how to download the historical stock price data of the **S&P 500** constituents using the `yfinance` library and clean the data by removing columns with missing values.

---

## 🔽 **Step 1: Install and Import `yfinance`**

To begin, you need to install the `yfinance` library, which can be done via `pip`. Open your terminal and run:

```bash
pip install yfinance


In [7]:
import yfinance as yf

df=yf.download(SP_500_DF.Symbol.tolist(),period='1y',auto_adjust=False) 
df

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')
['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")')


Price        Adj Close                                                  \
Ticker               A        AAPL        ABBV        ABNB         ABT   
Date                                                                     
2024-03-08  146.851044  169.927628  172.563904  164.910004  118.544174   
2024-03-11  146.275040  171.938141  173.316498  162.990005  117.789551   
2024-03-12  146.821259  172.415878  174.561157  166.669998  118.348167   
2024-03-13  146.801392  170.325760  173.538437  164.759995  117.760155   
2024-03-14  144.209381  172.186951  174.831314  166.440002  116.476311   
...                ...         ...         ...         ...         ...   
2025-03-03  126.330002  238.029999  211.479996  140.979996  140.220001   
2025-03-04  123.059998  235.929993  207.759995  141.419998  137.580002   
2025-03-05  126.510002  235.740005  211.539993  142.580002  137.710007   
2025-03-06  126.550003  235.330002  210.750000  134.770004  135.080002   
2025-03-07  126.699997  239.070007  214.289993  134.389999  137.139999   

Price                                                                 ...  \
Ticker           ACGL         ACN        ADBE         ADI        ADM  ...   
Date                                                                  ...   
2024-03-08  83.308220  373.546021  551.690002  192.740906  52.810596  ...   
2024-03-11  85.875641  368.656555  560.419983  194.973846  52.906952  ...   
2024-03-12  86.455696  375.827789  579.140015  195.947678  54.988155  ...   
2024-03-13  86.531769  374.089294  573.549988  195.357468  56.144379  ...   
2024-03-14  86.560295  373.585510  570.450012  191.255569  56.086571  ...   
...               ...         ...         ...         ...        ...  ...   
2025-03-03  92.980003  344.989990  440.720001  227.540009  46.580002  ...   
2025-03-04  91.870003  345.779999  446.029999  226.179993  45.820000  ...   
2025-03-05  91.349998  348.260010  451.239990  229.470001  45.799999  ...   
2025-03-06  91.379997  341.839996  444.779999  222.770004  48.060001  ...   
2025-03-07  91.000000  342.179993  449.399994  225.860001  49.439999  ...   

Price       Volume                                                          \
Ticker         WTW       WY     WYNN       XEL       XOM      XYL      YUM   
Date                                                                         
2024-03-08  354800  3734300  1137600   8848600  16274600   761500  1365000   
2024-03-11  608500  2202100  1943400   9838100  17577300  1019900  1711300   
2024-03-12  641900  3330900  1436800   8809700  14957200  1029700  1464800   
2024-03-13  530100  2508500  1138400   5420000  17122200   872500  3361400   
2024-03-14  482900  4124900  1882700  11860000  22121000  1335500  3091900   
...            ...      ...      ...       ...       ...      ...      ...   
2025-03-03  535400  6490600  2123200   5015600  18706900  1318000  1913300   
2025-03-04  470100  4294100  2272300   4212100  19394900  1819000  3969000   
2025-03-05  629100  3884900  2448900   6975000  24822200  1521100  2234300   
2025-03-06  636800  3997300  1797200   5262300  17150200  1504700  2020200   
2025-03-07  463800  5394200  1889600   3849600  19616400  1282800  3411300   

Price                                  
Ticker          ZBH     ZBRA      ZTS  
Date                                   
2024-03-08  1395200   328600  6518000  
2024-03-11  1079800   268800  3383500  
2024-03-12  1073000   316100  2598400  
2024-03-13   847000   286200  5947400  
2024-03-14  1203900   375300  6432600  
...             ...      ...      ...  
2025-03-03  1653200   586800  3020200  
2025-03-04  1233200   848400  3013700  
2025-03-05  2498400   578200  3667900  
2025-03-06  1081400   469000  3011500  
2025-03-07  1792900  1178300  3296800  

[250 rows x 3018 columns]

In [8]:
from datetime import date
from plotly import graph_objects as go
from plotly.subplots import make_subplots

# Get NVDA OHLCV
df_NVDA = df.reorder_levels(['Ticker', 'Price'], axis=1)['NVDA']

# Create subplots: 1 row, 2 columns (candlestick chart + volume plot)
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.15,  # Increased spacing between plots
                    subplot_titles=('NVDA Candlestick Chart', 'NVDA Volume'),
                    row_heights=[0.8, 0.2])

# Plot Candlestick Chart in the first subplot
fig.add_trace(go.Ohlc(x=df_NVDA.index,
                     open=df_NVDA['Open'],
                     high=df_NVDA['High'],
                     low=df_NVDA['Low'],
                     close=df_NVDA['Close']),
              row=1, col=1)

# Plot Volume Chart in the second subplot
fig.add_trace(go.Bar(x=df_NVDA.index,
                     y=df_NVDA['Volume'],
                     name='Volume'),
              row=2, col=1)

# Format Title and Layout
fig.update_layout(
    title={'text': 'NVDA Candlestick and Volume Chart', 'x': 0.5, 'font': dict(size=24)},
    xaxis_title='Date',  # X-axis title for the entire figure
    yaxis_title='Price (USD)',
    xaxis_rangeslider_visible=False,  # Hide range slider
    showlegend=False,
    height=800  # Set figure height to avoid squeezing
)

# Show the plot
fig.show()


## Adjusted Close Price for Analysis

For the rest of our analysis, we will focus on the **Adjusted Close** price of the asset. The Adjusted Close price accounts for stock splits and dividend payments, providing a more accurate reflection of the actual value of an asset over time.

### Why Not Total Return Index?
In a more comprehensive analysis, the **Total Return Index** would be preferred, as it incorporates both price appreciation and the reinvestment of dividends. This gives a more complete picture of the investment’s performance.

However, since Yahoo Finance does not provide a Total Return Index, we will proceed with the **Adjusted Close** price as a proxy for the total returns.


In [9]:
df = df['Adj Close']
df

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2024-03-08,146.851044,169.927628,172.563904,164.910004,118.544174,83.308220,373.546021,551.690002,192.740906,52.810596,...,270.343811,33.928585,99.301155,49.132404,104.841194,125.669731,136.874863,125.658829,282.589996,180.237778
2024-03-11,146.275040,171.938141,173.316498,162.990005,117.789551,85.875641,368.656555,560.419983,194.973846,52.906952,...,268.625183,34.152565,102.069679,50.952480,105.460304,124.779724,137.384842,126.947723,280.230011,181.553757
2024-03-12,146.821259,172.415878,174.561157,166.669998,118.348167,86.455696,375.827789,579.140015,195.947678,54.988155,...,271.598145,33.568268,101.881813,49.979851,104.783157,127.034424,138.032166,127.116295,285.440002,179.436325
2024-03-13,146.801392,170.325760,173.538437,164.759995,117.760155,86.531769,374.089294,573.549988,195.357468,56.144379,...,271.519165,33.655910,101.179802,49.594643,105.953651,126.599304,135.697937,125.510094,284.500000,174.370346
2024-03-14,144.209381,172.186951,174.831314,166.440002,116.476311,86.560295,373.585510,570.450012,191.255569,56.086571,...,270.630188,33.363758,99.864746,50.497929,107.830307,126.539955,134.521042,124.657448,281.769989,172.045166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-03,126.330002,238.029999,211.479996,140.979996,140.220001,92.980003,344.989990,440.720001,227.540009,46.580002,...,340.359985,31.171867,88.930000,71.900002,107.760002,129.779999,156.929993,105.379997,306.839996,170.529999
2025-03-04,123.059998,235.929993,207.759995,141.419998,137.580002,91.870003,345.779999,446.029999,226.179993,45.820000,...,333.940002,30.576038,87.099998,70.029999,107.540001,127.320000,158.410004,103.989998,297.920013,169.899994
2025-03-05,126.510002,235.740005,211.539993,142.580002,137.710007,91.349998,348.260010,451.239990,229.470001,45.799999,...,334.500000,30.715065,90.669998,68.730003,105.440002,129.669998,159.369995,104.750000,304.000000,167.000000


## 🧪 **Melting the DataFrame with `melt()`**

The `melt()` function in pandas is used to **transform the data from wide format to long format**. This is useful when you want to restructure the data to make it more suitable for analysis or visualization.

### 🏆 **Why Use `melt()`?**

- **Reshaping Data**: This transformation is typically used when you have data in a wide format (with multiple columns) and want to convert it into a long format where each row represents a single observation of an asset over time.
  
- **Easier Analysis & Plotting**: The long format is often more convenient for time-series analysis and creating visualizations, especially when plotting multiple series over the same x-axis (like dates or time periods).

This process is particularly useful for **financial data analysis**, where you may want to stack different assets (like stocks or assets from different time periods) in a single column for easier analysis.


In [77]:
df_long = df.melt(ignore_index=False, value_name='Adj Close')
df_long

,Ticker,Adj Close
Date,,
2024-03-08,A,146.851044
2024-03-11,A,146.275040
2024-03-12,A,146.821259
2024-03-13,A,146.801392
2024-03-14,A,144.209381
...,...,...
2025-03-03,ZTS,170.529999
2025-03-04,ZTS,169.899994
2025-03-05,ZTS,167.000000


In [78]:
df_long = df_long.reset_index().merge(SP_500_DF[['Symbol', 'GICS Sector', 'GICS Sub-Industry']], left_on='Ticker', right_on='Symbol').drop('Ticker',axis=1)
df_long


,Date,Adj Close,Symbol,GICS Sector,GICS Sub-Industry
0,2024-03-08,146.851044,A,Health Care,Life Sciences Tools & Services
1,2024-03-11,146.275040,A,Health Care,Life Sciences Tools & Services
2,2024-03-12,146.821259,A,Health Care,Life Sciences Tools & Services
3,2024-03-13,146.801392,A,Health Care,Life Sciences Tools & Services
4,2024-03-14,144.209381,A,Health Care,Life Sciences Tools & Services
...,...,...,...,...,...
125745,2025-03-03,170.529999,ZTS,Health Care,Pharmaceuticals
125746,2025-03-04,169.899994,ZTS,Health Care,Pharmaceuticals
125747,2025-03-05,167.000000,ZTS,Health Care,Pharmaceuticals
125748,2025-03-06,167.250000,ZTS,Health Care,Pharmaceuticals


In [84]:
import ipywidgets as widgets
import plotly.express as px
from IPython.display import display, clear_output

# Create the Dropdown widget for selecting Sub-Industry for YTD plot
sub_industry_dropdown_ytd = widgets.Dropdown(
    options=df_long['GICS Sub-Industry'].unique(),  # Get all unique Sub-Industries
    description='Sub-Industry:',
)

# Function to filter data, calculate start price, pct change, and plot graph
def filter_and_plot_ytd(sub_industry):
    # Filter the dataframe for the selected Sub-Industry
    filtered_df0 = df_long[df_long['GICS Sub-Industry'] == sub_industry].copy()  # Use .copy() to avoid the SettingWithCopyWarning

    # Get Start Price for each stock symbol
    filtered_df0['start_price'] = filtered_df0.groupby('Symbol')['Adj Close'].transform('first')

    # Get Percentage Change (Pct Change)
    filtered_df0['pct_change'] = (filtered_df0['Adj Close'] - filtered_df0['start_price']) / filtered_df0['start_price']

    # Plotting the data
    fig0 = px.line(filtered_df0, x='Date', y='pct_change', color='Symbol', height=600)

    # Adjusting the appearance of the plot
    fig0.update_traces(line=dict(width=3))
    fig0.update_layout(
        title={'text': f'{sub_industry} Stocks YTD Performance Comparison', 'x': 0.5, 'font': dict(size=24)},
        yaxis_tickfont=dict(size=20),
        xaxis_tickfont=dict(size=20),
        xaxis_title=dict(text='Date', font=dict(size=20)),
        yaxis_title=dict(text='Return', font=dict(size=20)),
        legend=dict(font=dict(size=20)),
        yaxis_tickformat=".0%",  # Format y-axis as percentage
        xaxis=dict(
            dtick="M1",  # Set x-axis ticks to each month
            tickformat="%b"  # Format x-axis tick labels as month abbreviations
        )
    )

    # Display the plot
    fig0.show()

# Attach the filtering and plotting function to the dropdown change
def update_ytd_plot(change):
    # Clear the output before updating the plot (this is needed to update the plot)
    clear_output(wait=True)
    # Call the function to update the plot with the selected sub-industry
    filter_and_plot_ytd(change.new)
    # Display the dropdown again after plot is updated
    display(sub_industry_dropdown_ytd)

# Observe the change in dropdown value and trigger the update
sub_industry_dropdown_ytd.observe(update_ytd_plot, names='value')

# Display the dropdown widget
display(sub_industry_dropdown_ytd)


Dropdown(description='Sub-Industry:', index=7, options=('Life Sciences Tools & Services', 'Technology Hardware…

## Introduction to Technical Analysis 📊

**Technical Analysis** 📉 is a method of evaluating and predicting the future price movements of an asset based on historical price data, trading volume, and other market indicators. Unlike fundamental analysis, which focuses on the intrinsic value of an asset, technical analysis primarily relies on charts and patterns to assess market behavior.

### Key Concepts in Technical Analysis:
- **Charts** 📈: The foundation of technical analysis is charting. Common chart types include line charts, bar charts, and candlestick charts.
- **Trends** 🔽🔼: Identifying trends (uptrend, downtrend, or sideways) is essential in technical analysis. Understanding market sentiment is key to making predictions.
- **Support and Resistance** 🔒📈: Support refers to a price level where an asset tends to find buying interest, while resistance is a level where selling interest is likely to emerge.
- **Indicators and Oscillators** 📉📊: These are mathematical calculations based on the asset’s price and volume. Examples include Moving Averages, Relative Strength Index (RSI), Bollinger Bands, and MACD.

In technical analysis, it is believed that all known information is already reflected in the price of an asset, and therefore, historical price patterns can help forecast future price movements.

In the following section, we will explore one technical indicator (RSI) and how they can be used to assess and predict stock market trends.


### 📉 **Relative Strength Index (RSI) Explanation**

The **Relative Strength Index (RSI)** is a momentum oscillator that measures the speed and change of price movements. RSI is used to identify overbought or oversold conditions in a market, which can signal potential price reversals. 

#### **RSI Formula**

The formula for calculating the RSI is as follows:

$
RSI = 100 - \frac{100}{1 + RS}
$

Where:
- $ RS = \frac{\text{Average Gain}}{\text{Average Loss}} $

- **Average Gain** is the average of the gains over a specified period (default is 14 periods).
- **Average Loss** is the average of the losses over the same period.

#### **Steps to Calculate RSI:**
1. **Calculate the daily price changes**:
   $
   \Delta P_t = P_t - P_{t-1}
   $
2. **Separate the gains and losses**:
   - If \($\Delta P_t > 0$\), the gain is \($\Delta P_t$\), otherwise, the loss is \($-\Delta P_t$\).
3. **Calculate the average gains and losses** over the window (default 14 periods).
4. **Calculate the relative strength (RS)** as the ratio of the average gain to the average loss.
5. **Apply the RSI formula**.

The RSI typically uses a **14-day window** by default.

---

### 📊 **Code Example for RSI Calculation**

The following Python code snippet demonstrates how to compute the RSI for a stock using the **`ta`** library:

```python
import ta

# Select the QCOM stock data
df_QCOM = df['QCOM']

# Calculate RSI with a 14-day window
df_QCOM['RSI'] = ta.momentum.RSIIndicator(df_QCOM['Adj Close'], window=14).rsi() #the default window length is also usually 14
```
Below is a dynamic Figure that displays the top 5 companies, ordered by RSI, once a sub-industry has been selected.

In [86]:
import ipywidgets as widgets
import plotly.express as px
import ta
from IPython.display import display, clear_output

# Dropdown for Sub-Industry selection
sub_industry_dropdown = widgets.Dropdown(
    options=df_long['GICS Sub-Industry'].unique(),  
    description='Sub-Industry:',
)

# Function to filter data, calculate RSI, and plot
def filter_and_plot_data(sub_industry):
    df_sub = df_long[df_long['GICS Sub-Industry'] == sub_industry]
    df_rsi = df_sub.groupby('Symbol').apply(lambda l: ta.momentum.RSIIndicator(l['Adj Close']).rsi().iloc[-1]).T.sort_values().to_frame('rsi')# default RSI is always 14 days
    top_5_rsi = df_rsi.nlargest(5, 'rsi')

    # Plot
    fig = px.bar(top_5_rsi.reset_index(), y='Symbol', x='rsi', color='Symbol', height=1200, 
                 title=f'Top 5 RSI Stocks in {sub_industry}')
    fig.update_traces(texttemplate='%{y} %{x:.2f}', textposition='inside', textfont=dict(size=16))
    fig.update_layout(
        height=800, width=800,
        title={'text': f'Top 5 RSI Stocks in {sub_industry}', 'x': 0.5, 'font': dict(size=28, color='blue')},
        yaxis=dict(automargin=True, tickmode='linear', tickfont=dict(size=26)),
        xaxis_tickfont=dict(size=26),
        xaxis_title=dict(text="RSI", font=dict(size=26)),
        yaxis_title=dict(text="Stock Symbol", font=dict(size=26)),
    )
    fig.show()

# Update function for dropdown
def update_plot(change):
    clear_output(wait=True)
    filter_and_plot_data(change.new)
    display(sub_industry_dropdown)

# Observe dropdown change
sub_industry_dropdown.observe(update_plot, names='value')

# Display dropdown
display(sub_industry_dropdown)


/var/folders/gm/fn54mj5d4b51_7l5j667v4n00000gn/T/ipykernel_1423/3534313332.py:15: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



Dropdown(description='Sub-Industry:', index=9, options=('Life Sciences Tools & Services', 'Technology Hardware…

# 📊 RSI Example: Hewlett Packard Enterprise (HPE) Stocks 🏢

As an example of how RSI works, let's apply it to **Hewlett Packard Enterprise** stocks. RSI helps us identify whether a stock is overbought or oversold. 

We will calculate the **latest RSI value** for Hewlett Packard Enterprise and analyze its performance based on the momentum indicator. 📉📈


In [90]:
# Select the 'HPE' column and rename it to 'Adj Close'
df_HPE = df['HPE'].to_frame()  # Convert to DataFrame
df_HPE.rename(columns={'HPE': 'Adj Close'}, inplace=True)

# Calculate RSI for the 'Adj Close' column
df_HPE['RSI'] = ta.momentum.RSIIndicator(df_HPE['Adj Close'], window=14).rsi()  # default RSI is 14 days

# Display the result
df_HPE

,Adj Close,RSI
Date,,
2024-03-08,17.516361,NaN
2024-03-11,17.565046,NaN
2024-03-12,17.574781,NaN
2024-03-13,17.428728,NaN
2024-03-14,17.193340,NaN
...,...,...
2025-03-03,19.030001,27.067950
2025-03-04,18.840000,25.966371
2025-03-05,18.889999,26.810487


In [91]:
df_HPE['RSI_BECAME_OVERSOLD'] = (df_HPE['RSI'] < 30) & (df_HPE['RSI'].shift(1) > 30 )


In [89]:
df_HPE_long = df_HPE.melt(ignore_index=False, var_name='metric', value_name='value')
fig = px.line(df_HPE_long, y='value', facet_row='metric', color='metric')
fig.update_yaxes(matches=None)
fig.show()


In [92]:
# Create a cumulative sum to group oversold events
df_HPE['OVERSOLD_GROUP'] = df_HPE['RSI_BECAME_OVERSOLD'].cumsum()

# Filter the DataFrame for oversold events
df_HPE_OVERSOLD = df_HPE[df_HPE['OVERSOLD_GROUP'] > 0]

# Get the top 3 oversold groups
top_2_oversold_groups = df_HPE_OVERSOLD['OVERSOLD_GROUP'].value_counts().nlargest(2).index

# Filter only top 3 oversold groups
df_top_2_oversold = df_HPE_OVERSOLD[df_HPE_OVERSOLD['OVERSOLD_GROUP'].isin(top_2_oversold_groups)]

# Group by oversold group and take the first 10 entries from each group
df_top_2_oversold_10 = df_top_2_oversold.groupby('OVERSOLD_GROUP').head(10)

# Create the figure using Plotly Express
fig = px.line(df_top_2_oversold_10, 
              y='Adj Close', 
              color='OVERSOLD_GROUP',  # Color by oversold group
              title="Top 2 Oversold Events",
              height=600,
              template="plotly_dark")  # Use a dark theme

# Show the plot
fig.show()

# 📊 Get Latest RSI for All S&P 500 Stocks and Keep Sector Info 🏢

Stay updated with the **Relative Strength Index (RSI)** for all **S&P 500 stocks** 📈 and keep track of their **sector information** 🏙️.


In [93]:
df_rsi = df_long.groupby(['Symbol', 'GICS Sector'])['Adj Close'].apply(lambda x: ta.momentum.RSIIndicator(x, window=14).rsi().iloc[-1])
df_rsi = df_rsi.to_frame('Latest RSI').reset_index()

# Sort by RSI values in ascending order and drop NaNs
df_rsi = df_rsi.sort_values(by='Latest RSI', ascending=True).dropna().reset_index(drop=True)

df_rsi


,Symbol,GICS Sector,Latest RSI
0,HPE,Information Technology,14.919546
1,DECK,Consumer Discretionary,22.028820
2,CZR,Consumer Discretionary,22.283987
3,VMC,Materials,22.837085
4,DAY,Industrials,23.003162
...,...,...,...
498,JKHY,Financials,78.343400
499,GILD,Health Care,78.477748
500,YUM,Consumer Discretionary,80.344443
501,BF.B,Consumer Staples,100.000000


## 🧐 Inspecting the RSI Data  

After reviewing the RSI values, we noticed that the tickers **BF.B** & **BRK.B** have an **RSI of 100 for NA values**. This could lead to misleading insights.  

🔍 **Key takeaway:** Always clean your data before analysis! If not, you risk the classic **"garbage in, garbage out"** scenario.  

### 🛠️ Fixing the Issue  
To ensure accurate results, we'll remove these anomalies before proceeding. Let's keep our data **clean and reliable!** 🚀  


In [94]:
# Remove rows where RSI is 100 and Symbol is BF.B or BRK.B
df_rsi_cleaned = df_rsi[~((df_rsi['Latest RSI'] == 100) & (df_rsi['Symbol'].isin(['BF.B', 'BRK.B'])))]

# Display cleaned DataFrame
df_rsi_cleaned


,Symbol,GICS Sector,Latest RSI
0,HPE,Information Technology,14.919546
1,DECK,Consumer Discretionary,22.028820
2,CZR,Consumer Discretionary,22.283987
3,VMC,Materials,22.837085
4,DAY,Industrials,23.003162
...,...,...,...
496,ABBV,Health Care,76.741734
497,VZ,Communication Services,76.991988
498,JKHY,Financials,78.343400
499,GILD,Health Care,78.477748


In [95]:
# Sort the dataframe by RSI and get the top 10 and bottom 10
top_10_rsi = df_rsi_cleaned.nlargest(10, 'Latest RSI')
bottom_10_rsi = df_rsi_cleaned.nsmallest(10, 'Latest RSI')

# Concatenate top and bottom RSI data
df_rsi_top_bottom = pd.concat([top_10_rsi, bottom_10_rsi])

# Create a horizontal bar chart for the top and bottom RSI values
fig = px.bar(df_rsi_top_bottom,
             y='Symbol', 
             x='Latest RSI', 
             color='Symbol',
             orientation='h',  # Horizontal bars
             title="Top 10 and Bottom 10 RSI Values (Overbought and Oversold)",
             labels={'Symbol': 'Stock Symbol', 'Latest RSI': 'RSI Value'})

# Add text to the bars
fig.update_traces(texttemplate='%{y} %{x:.2f}', textposition='inside', textfont=dict(size=16))

# Adjusting the appearance of the plot
fig.update_layout(
    height=1200, 
    width=1000,  # Set desired height and width
    title={'text': 'Top and Bottom 10 RSI Stocks (Overbought and Oversold)', 'x': 0.5, 'font': dict(size=28, color='blue')},
    yaxis=dict(automargin=True, tickmode='linear'),  # Force all labels to show
    yaxis_tickfont=dict(size=26),  # Increase font size of y-axis labels
    xaxis_tickfont=dict(size=26),  # Increase font size of x-axis labels
    xaxis_title=dict(text="RSI", font=dict(size=26)),  # Font size for x-axis title
    yaxis_title=dict(text="Stock Symbol", font=dict(size=26))  # Font size for y-axis title
)

# Display the plot
fig.show()

# 📊 Part 4: **Fundamental Analysis with Yahoo Finance** 📈

In this section, we'll dive into calculating important **fundamental metrics** for stocks using the data from **Yahoo Finance**.

---

### 🔢 **P/E Ratio Formula:**

The **Price-to-Earnings (P/E) Ratio** is one of the most commonly used metrics to assess whether a stock is **overvalued** or **undervalued**. It is calculated as:

$
\text{P/E Ratio} = \frac{\text{Latest Stock Price (Adj Close)}}{\text{Trailing 12-Month Earnings (EPS)}}
$

Where:
- 📅 **Latest Stock Price (Adj Close)**: The most recent **adjusted closing price** of the stock.
- 🧮 **Trailing 12-Month Earnings (EPS)**: The **earnings per share (EPS)** for the last 12 months.

---

### 🏷️ **Other Fundamental Metrics**:

1. **Forward P/E**: 
   - A variation of the P/E ratio based on **forecasted earnings** for the next 12 months.

2. **Price-to-Book (P/B) Ratio**:
   - Measures a company's **market value** relative to its **book value**.

   $
   \text{P/B Ratio} = \frac{\text{Market Price per Share}}{\text{Book Value per Share}}
   $

3. **EPS (TTM)**:
   - **Earnings per Share (EPS)** for the **trailing twelve months**.

4. **Debt-to-Equity Ratio (D/E)**:
   - Indicates the **financial leverage** of a company, calculated as:

   $
   \text{Debt-to-Equity Ratio} = \frac{\text{Total Debt}}{\text{Total Equity}}
   $

5. **Market Cap**:
   - The total market value of a company’s **outstanding shares**.

   $
   \text{Market Cap} = \text{Share Price} \times \text{Shares Outstanding}
   $

6. **Price-to-Sales Ratio (P/S)**:
   - Compares a company’s stock price to its **total revenue**.

   $
   \text{P/S Ratio} = \frac{\text{Market Cap}}{\text{Total Revenue}}
   $

7. **Profit Margin**:
   - Measures the percentage of revenue that exceeds the **cost of goods sold (COGS)**.

   $
   \text{Profit Margin} = \frac{\text{Net Income}}{\text{Revenue}} \times 100
   $

8. **Return on Equity (ROE)**:
   - Measures a company’s **profitability** relative to shareholders' equity.
   
   $
   \text{ROE} = \frac{\text{Net Income}}{\text{Shareholders' Equity}} \times 100
   $

9. **Earnings Growth**:
   - The percentage growth in a company’s **earnings** over a specific period.

10. **Revenue Growth**:
    - The percentage growth in a company’s **revenue** over a specified period.

11. **Total Cash**:
    - The total amount of **cash** and **cash equivalents** the company holds.

12. **Total Debt**:
    - The total amount of **debt** the company owes.

13. **Book Value**:
    - The **net asset value** of the company, calculated as:

    $
    \text{Book Value} = \text{Total Assets} - \text{Total Liabilities}
    $

---

### 📉 **Visualizing Fundamental Metrics**:

We can compare multiple companies based on these fundamental metrics by creating **visualizations**. For example, we can use **strip plots** to compare different stocks across the various **financial ratios**.

---

Feel free to copy and paste the above markdown into your Jupyter notebook for a neat and organized presentation of **Fundamental Analysis**! 💼💡


In [37]:
def get_fundamental_data(symbol, last_price):
    # Print the stock symbol and its last price to indicate the start of data fetching
    print(f"Fetching data for {symbol}...")
    print(f"Last Price: {last_price}")
    
    """
    Fetches fundamental data for a single stock symbol.

    Args:
        symbol (str): The stock ticker symbol. (e.g., 'AAPL', 'GOOG', etc.)
        last_price (float): The last traded price of the stock (typically from the Adj Close).

    Returns:
        pandas.Series: A Series containing the fundamental data for the given stock,
                       or None if data retrieval fails due to errors.
    """

    # Use the Yahoo Finance library to fetch data for the given stock symbol
    ticker = yf.Ticker(symbol)
    
    try:
        # Fetch fundamental financial data using Yahoo Finance's API
        data = {
            # These are different fundamental financial metrics we are fetching from the ticker's information:
            #"Trailing P/E": ticker.info.get('trailingPE', None),  # Commented out because it's not used
            "Forward P/E": ticker.info.get('forwardPE', None),  # Forward Price-to-Earnings ratio
            "Price-to-Book": ticker.info.get('priceToBook', None),  # Price-to-Book ratio
            "EPS (TTM)": ticker.info.get('trailingEps', None),  # Earnings Per Share (Trailing 12 Months)
            "Debt-to-Equity": ticker.info.get('debtToEquity', None),  # Debt-to-Equity ratio
            #"Current Ratio": ticker.info.get('currentRatio', None),  # Commented out
            #"Quick Ratio": ticker.info.get('quickRatio', None),  # Commented out
            "Market Cap": ticker.info.get('marketCap', None),  # Market Capitalization
            "Total Revenue": ticker.info.get('totalRevenue', None),  # Total Revenue
            "Price-to-Sales": ticker.info.get('priceToSalesTrailing12Months', None),  # Price-to-Sales ratio
            #"Gross Profit": ticker.info.get('grossProfits', None),  # Commented out
            #"Operating Margin": ticker.info.get('operatingMargin', None),  # Commented out
            "Profit Margin": ticker.info.get('profitMargins', None),  # Profit Margin
            "Return on Equity": ticker.info.get('returnOnEquity', None),  # Return on Equity
            "Earnings Growth": ticker.info.get('earningsGrowth', None),  # Earnings Growth
            "Revenue Growth": ticker.info.get('revenueGrowth', None),  # Revenue Growth
            "Total Cash": ticker.info.get('totalCash', None),  # Total Cash on hand
            "Total Debt": ticker.info.get('totalDebt', None),  # Total Debt
            "Book Value": ticker.info.get('bookValue', None)  # Book Value of the company
        }

        # Convert the data dictionary into a pandas Series for easy manipulation
        fundamentals = pd.Series(data, name=symbol)
        
        # Calculate the P/E ratio by dividing the last price by the trailing 12-month EPS
        fundamentals['P/E Ratio'] = last_price / fundamentals['EPS (TTM)']
        
        # Return the full Series containing all the fetched fundamental data
        return fundamentals

    # Handle exceptions in case data retrieval fails (e.g., missing data, bad symbol)
    except (KeyError, IndexError, TypeError) as e:
        # Print an error message if any error occurs while fetching data
        print(f"Error fetching data for {symbol}: {e}")
        
        # Return None to indicate the failure to fetch data
        return None


In [66]:
#looking at the market 
df_CE_fundamentals = df[['HPE','MSI','FFIV']].iloc[-1].to_frame('Last Price').reset_index()
df_CE_fundamentals = df_CE_fundamentals.apply(lambda l:get_fundamental_data(l['Ticker'],l['Last Price']),axis=1).set_index(df_CE_fundamentals['Ticker'])
df_CE_fundamentals = df_CE_fundamentals.melt(value_name='Value',var_name='Metric',ignore_index=False)

df_CE_fundamentals

Fetching data for HPE...
Last Price: 15.8100004196167
Fetching data for MSI...
Last Price: 423.1300048828125
Fetching data for FFIV...
Last Price: 275.6300048828125


,Metric,Value
Ticker,,
HPE,Forward P/E,7.528572e+00
MSI,Forward P/E,2.880395e+01
FFIV,Forward P/E,1.789805e+01
HPE,Price-to-Book,8.229660e-01
MSI,Price-to-Book,4.151589e+01
FFIV,Price-to-Book,4.951852e+00
HPE,EPS (TTM),2.080000e+00
MSI,EPS (TTM),9.220000e+00
FFIV,EPS (TTM),1.004000e+01


In [69]:
fig = px.strip(
    df_CE_fundamentals.reset_index(),
    y='Ticker',
    x='Value',
    color='Ticker',
    facet_col='Metric',  # Use facet_col for horizontal titles
    facet_col_wrap=3
)

# Update layout
fig.update_layout(
    title={'text':'Stock Comparison by Metric','x':0.5,'font':dict(size=28,color='blue')},
    height=1000,  # Adjust height to fit all facets
    width=1000,  # Adjust width for better layout
    margin=dict(t=50, l=50, r=50, b=50),  # Adjust margins,
)

# Force all y-axis ticks to show
fig.update_yaxes(
    matches=None,
    tickmode='array',  # Use array mode to show specific ticks
    tickvals=df_CE_fundamentals.index.unique(),  # Set tick values explicitly
    tickfont=dict(size=16) # Increase y-axis tick font size
)

# Update traces for square shape and potentially larger size
fig.update_traces(
    marker_symbol='square',  # Make dots square
    marker_size=15          # Increase size further if needed
)

# Ensure x-axes are independent and complete
fig.update_xaxes(matches=None, tickfont=dict(size=16)) # Increase x-axis tick font size

# Increase and center tile font (facet titles)
for annotation in fig.layout.annotations:
    annotation.font.size = 15  # Increase tile font size
    annotation.xanchor = 'center'  # Center the title horizontally

# Increase metric font (axis titles)
fig.update_xaxes(title_font=dict(size=15)) # Increase x-axis title font size
fig.update_yaxes(title_font=dict(size=15)) # Increase y-axis title font size

fig.for_each_xaxis(lambda x: x.update(
    tickmode='array',
    matches=None, showticklabels=True
))

fig.for_each_yaxis(lambda x: x.update(
    tickmode='array',tickvals=df_CE_fundamentals.index.unique(),
    matches=None,categoryorder='category ascending'

))

fig.show()